In [1]:
from Bio.PDB import *
from Bio.PDB.DSSP import DSSP
from Bio.PDB.DSSP import dssp_dict_from_pdb_file
from Bio import SeqIO
from Bio.SeqUtils import seq1
import Bio

from Bio import pairwise2
from Bio.pairwise2 import format_alignment
from Bio.SubsMat import MatrixInfo as matlist



import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy.linalg as linalg
from scipy import sparse
import scipy.signal
from snf import *

import pandas as pd

from difflib import SequenceMatcher


/Users/boyanxu/opt/anaconda3/lib/python3.9/site-packages/Bio/SubsMat/__init__.py:126: BiopythonDeprecationWarning: Bio.SubsMat has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.substitution_matrices as a replacement, and contact the Biopython developers if you still need the Bio.SubsMat module.
  warnings.warn(


In [65]:
df = pd.read_csv('LRRPredictor/col-0-nlrome.txt', delimiter = '\t')


In [66]:
tairIDs = ['.'.join(id.split('_')[1:]) for id in set(df['annotation'])]

In [67]:
tairIDs

['AT5G45050.1',
 'AT1G10920.1',
 'AT5G38350.1',
 'AT1G12290.2',
 'AT1G50180.1',
 'AT1G58400.1',
 'AT5G44510.1',
 'AT1G17600.1',
 'AT3G07040.1',
 'AT3G50950.1',
 'AT5G41740.2',
 'AT5G51630.2',
 'AT1G61310.1',
 'AT3G44480.1',
 'AT1G56520.2',
 'AT1G15890.1',
 'AT4G16920.1',
 'AT1G72840.1',
 'AT4G09430.1',
 'AT4G12020.1',
 'AT3G44670.2',
 'AT1G10920.2',
 'AT1G12210.1',
 'AT5G45200.1',
 'AT1G12290.1',
 'AT4G16960.1',
 'AT5G43470.2',
 'AT5G66910.1',
 'AT1G58807.1',
 'AT4G19520.1',
 'AT1G57650.2',
 'AT5G48620.1',
 'AT5G18370.1',
 'AT4G19050.1',
 'AT4G36140.1',
 'AT5G49140.1',
 'AT5G47260.1',
 'AT3G51570.1',
 'AT1G56520.1',
 'AT5G40060.1',
 'AT1G63730.1',
 'AT4G16950.2',
 'AT5G46470.1',
 'AT3G46530.1',
 'AT5G40910.1',
 'AT3G44630.2',
 'AT1G12220.2',
 'AT3G50950.2',
 'AT1G59124.1',
 'AT1G65850.1',
 'AT5G47250.1',
 'AT5G17880.1',
 'AT1G63750.1',
 'AT1G56540.1',
 'AT5G58120.1',
 'AT5G22690.1',
 'AT1G61180.1',
 'AT1G61190.1',
 'AT5G44870.1',
 'AT4G33300.2',
 'AT4G19500.2',
 'AT4G36150.1',
 'AT1G51

In [58]:
from os import listdir
from os.path import isfile, join


allupIDs = set([f.split('-')[1]for f in listdir('afdb') if isfile(join('afdb', f)) and f.startswith('AF-') and f.endswith('pdb.gz')])

In [80]:
headers = {"Accept": "text/plain; format=flatfile"}
url = "https://rest.uniprot.org/uniprotkb/search?query=organism_id:3702+%s"%'AT1G58807.1'
r = requests.post(url, headers=headers)

r.text

# with open("temp_string/uniprot.txt", 'w') as file:
    # file.write(r.text)



'ID   DRL44_ARATH             Reviewed;        1017 AA.\nAC   P0DI16; Q8W3J8; Q8W3J9; Q94HW5; Q9SLU5; Q9SLU6;\nDT   16-MAY-2012, integrated into UniProtKB/Swiss-Prot.\nDT   16-MAY-2012, sequence version 1.\nDT   14-DEC-2022, entry version 82.\nDE   RecName: Full=Probable disease resistance protein RDL5;\nGN   Name=RDL5; OrderedLocusNames=At1g59124; ORFNames=T4M14.5;\nOS   Arabidopsis thaliana (Mouse-ear cress).\nOC   Eukaryota; Viridiplantae; Streptophyta; Embryophyta; Tracheophyta;\nOC   Spermatophyta; Magnoliopsida; eudicotyledons; Gunneridae; Pentapetalae;\nOC   rosids; malvids; Brassicales; Brassicaceae; Camelineae; Arabidopsis.\nOX   NCBI_TaxID=3702;\nRN   [1]\nRP   NUCLEOTIDE SEQUENCE [GENOMIC DNA] (ISOFORM 2).\nRC   STRAIN=cv. Columbia;\nRX   PubMed=10548732; DOI=10.1016/s0378-1119(99)00403-5;\nRA   Kato A., Suzuki M., Kuwahara A., Ooe H., Higano-Inaba K., Komeda Y.;\nRT   "Isolation and analysis of cDNA within a 300 kb Arabidopsis thaliana\nRT   genomic region located around th

In [81]:
r.text

['ID   DRL44_ARATH             Reviewed;        1017 AA.',
 'AC   P0DI16; Q8W3J8; Q8W3J9; Q94HW5; Q9SLU5; Q9SLU6;',
 'DT   16-MAY-2012, integrated into UniProtKB/Swiss-Prot.',
 'DT   16-MAY-2012, sequence version 1.',
 'DT   14-DEC-2022, entry version 82.',
 'DE   RecName: Full=Probable disease resistance protein RDL5;',
 'GN   Name=RDL5; OrderedLocusNames=At1g59124; ORFNames=T4M14.5;',
 'OS   Arabidopsis thaliana (Mouse-ear cress).',
 'OC   Eukaryota; Viridiplantae; Streptophyta; Embryophyta; Tracheophyta;',
 'OC   Spermatophyta; Magnoliopsida; eudicotyledons; Gunneridae; Pentapetalae;',
 'OC   rosids; malvids; Brassicales; Brassicaceae; Camelineae; Arabidopsis.',
 'OX   NCBI_TaxID=3702;',
 'RN   [1]',
 'RP   NUCLEOTIDE SEQUENCE [GENOMIC DNA] (ISOFORM 2).',
 'RC   STRAIN=cv. Columbia;',
 'RX   PubMed=10548732; DOI=10.1016/s0378-1119(99)00403-5;',
 'RA   Kato A., Suzuki M., Kuwahara A., Ooe H., Higano-Inaba K., Komeda Y.;',
 'RT   "Isolation and analysis of cDNA within a 300 kb Arabido

In [76]:
import gffpandas.gffpandas as gffpd

annotation = gffpd.read_gff3("temp_string/uniprot.txt")

In [78]:
annotation.df

,seq_id,source,type,start,end,score,strand,phase,attributes
P0DI16,UniProtKB,Chain,1,1017,.,.,.,ID=PRO_0000417428;Note=Probable disease resist...,NaN
P0DI16,UniProtKB,Domain,147,460,.,.,.,Note=NB-ARC,NaN
P0DI16,UniProtKB,Repeat,602,627,.,.,.,Note=LRR 1,NaN
P0DI16,UniProtKB,Repeat,649,674,.,.,.,Note=LRR 2,NaN
P0DI16,UniProtKB,Repeat,675,699,.,.,.,Note=LRR 3,NaN
P0DI16,UniProtKB,Repeat,768,791,.,.,.,Note=LRR 4,NaN
P0DI16,UniProtKB,Repeat,792,819,.,.,.,Note=LRR 5,NaN
P0DI16,UniProtKB,Repeat,841,865,.,.,.,Note=LRR 6,NaN
P0DI16,UniProtKB,Repeat,937,962,.,.,.,Note=LRR 7,NaN
P0DI16,UniProtKB,Coiled coil,25,52,.,.,.,Ontology_term=ECO:0000255;evidence=ECO:0000255,NaN


In [41]:
df = pd.read_csv(StringIO(r.text), delimiter='\t')
df

,Entry,Entry Name,Reviewed,Protein names,Gene Names,Organism,Length
0,A0A384LA06,A0A384LA06_ARATH,unreviewed,Auxin-responsive protein,AXX17_At5g41740 AN1_LOCUS24567 AT9943_LOCUS212...,Arabidopsis thaliana (Mouse-ear cress),186
1,F4JYI5,F4JYI5_ARATH,unreviewed,Disease resistance protein (TIR-NBS-LRR class)...,MUF8.2 MUF8_2 At5g41740 AT5G41740,Arabidopsis thaliana (Mouse-ear cress),1046
2,A0A1P8BEY3,A0A1P8BEY3_ARATH,unreviewed,Disease resistance protein (TIR-NBS-LRR class)...,MUF8.2 MUF8_2 At5g41740 AT5G41740,Arabidopsis thaliana (Mouse-ear cress),894
3,F4JYI4,F4JYI4_ARATH,unreviewed,Disease resistance protein (TIR-NBS-LRR class)...,MUF8.2 MUF8_2 At5g41740 AT5G41740,Arabidopsis thaliana (Mouse-ear cress),1114
4,Q0WUW2,Q0WUW2_ARATH,unreviewed,Disease resistance like protein,At5g41740,Arabidopsis thaliana (Mouse-ear cress),575


In [91]:
upIDs = []
headers = {"Accept": "text/plain; format=tsv"}
dups = []
for jj, tairid in enumerate(tairIDs):
    url = "https://rest.uniprot.org/uniprotkb/search?query=organism_id:3702+%s"%tairid
    r = requests.post(url, headers=headers)
    df = pd.read_csv(StringIO(r.text), delimiter='\t')['Entry']
    print(df[0], jj, tairid)
    entr_l = [entr for entr in df if entr in allupIDs]
    if len(entr_l)>1:
        print('more than one entry')
        dups.append(jj)
    if not len(entr_l):
        print('no entries found')
    upIDs.append(entr_l)
        

    


Q9FL92 0 AT5G45050.1
O04093 1 AT1G10920.1
Q9FF28 2 AT5G38350.1
P60839 3 AT1G12290.2
Q9SX38 4 AT1G50180.1
Q8W3K3 5 AT1G58400.1
Q9FI14 6 AT5G44510.1
F4I901 7 AT1G17600.1
Q39214 8 AT3G07040.1
Q38834 9 AT3G50950.1
F4JYI5 10 AT5G41740.2
A0A1P8BAZ5 11 AT5G51630.2
O64789 12 AT1G61310.1
F4J339 13 AT3G44480.1
Q8GY18 14 AT1G56520.2
Q9LMP6 15 AT1G15890.1
A0A1P8B6H3 16 AT4G16920.1
F4IEZ7 17 AT1G72840.1
F4JK03 18 AT4G09430.1
Q9SZ67 19 AT4G12020.1
F4J361 20 AT3G44670.2
O04093 21 AT1G10920.2
Q8L3R3 22 AT1G12210.1
A0A178UP78 23 AT5G45200.1
P60839 24 AT1G12290.1
F4JNB9 25 AT4G16960.1
Q8W4J9 26 AT5G43470.2
Q9FKZ0 27 AT5G66910.1
P0DI16 28 AT1G58807.1
more than one entry
F4JT82 29 AT4G19520.1
F4I847 30 AT1G57650.2
Q9FJK8 31 AT5G48620.1
F4JWM0 32 AT5G18370.1
Q8GYX0 33 AT4G19050.1
more than one entry
O65506 34 AT4G36140.1
Q9FH20 35 AT5G49140.1
Q9LVT3 36 AT5G47260.1
Q9SCZ2 37 AT3G51570.1
Q8GY18 38 AT1G56520.1
F4KFY1 39 AT5G40060.1
Q9CAD8 40 AT1G63730.1
F4JNB7 41 AT4G16950.2
F4KHH8 42 AT5G46470.1
Q9M667 43 AT

In [94]:
for ii in dups:
    print(tairIDs[ii])
    print(upIDs[ii])

AT1G58807.1
['P0DI16', 'F4IBE4']
AT4G19050.1
['Q8GYX0', 'P0CB16']
AT1G59124.1
['P0DI16', 'F4IBE4']
AT1G58807.2
['P0DI16', 'F4IBE4']
AT1G53350.1
['P0C8S1', 'Q3ECR3']
AT4G11170.1
['O82500', 'O82499']
AT1G64070.1
['F4I594', 'Q6DST1']
AT1G59218.1
['P0DI18', 'P0DI17']
AT1G58848.1
['P0DI17', 'P0DI18']
AT1G59218.2
['P0DI18', 'P0DI17']
AT1G58848.2
['P0DI18', 'P0DI17']


In [95]:

uniprot_d = {}
for ii, entr_l in enumerate(upIDs):
    if entr_l and not ii in dups:
        uniprot_d[tairIDs[ii]] = entr_l[0]


In [97]:
uniprot_d

{'AT5G45050.1': 'Q9FL92',
 'AT1G10920.1': 'O04093',
 'AT5G38350.1': 'Q9FF28',
 'AT1G12290.2': 'P60839',
 'AT1G50180.1': 'Q9SX38',
 'AT1G58400.1': 'Q8W3K3',
 'AT5G44510.1': 'Q9FI14',
 'AT1G17600.1': 'F4I901',
 'AT3G07040.1': 'Q39214',
 'AT3G50950.1': 'Q38834',
 'AT5G41740.2': 'F4JYI4',
 'AT5G51630.2': 'Q0WPW2',
 'AT1G61310.1': 'O64789',
 'AT3G44480.1': 'F4J339',
 'AT1G56520.2': 'F4I552',
 'AT1G15890.1': 'Q9LMP6',
 'AT4G16920.1': 'A0A1P8B6H3',
 'AT1G72840.1': 'F4IEZ7',
 'AT4G09430.1': 'F4JK03',
 'AT4G12020.1': 'Q9SZ67',
 'AT3G44670.2': 'F4J361',
 'AT1G10920.2': 'O04093',
 'AT1G12210.1': 'Q8L3R3',
 'AT5G45200.1': 'F4KD45',
 'AT1G12290.1': 'P60839',
 'AT4G16960.1': 'F4JNB9',
 'AT5G43470.2': 'Q8W4J9',
 'AT5G66910.1': 'Q9FKZ0',
 'AT4G19520.1': 'F4JT82',
 'AT1G57650.2': 'F4I847',
 'AT5G48620.1': 'Q9FJK8',
 'AT5G18370.1': 'F4JWM0',
 'AT4G36140.1': 'O65506',
 'AT5G49140.1': 'Q9FH20',
 'AT5G47260.1': 'Q9LVT3',
 'AT3G51570.1': 'Q9SCZ2',
 'AT1G56520.1': 'F4I552',
 'AT5G40060.1': 'F4KFY1',
 'AT1G63

In [98]:
for ii, upid in zip(dups, ["F4IBE4", "P0CB16", "P0DI16", "F4IBE4", "P0C8S1", "O82500", "F4I594", "P0DI18", "P0DI17", "P0DI18", "P0DI17"]):
    uniprot_d[tairIDs[ii]] = upid

In [99]:

uniprot_d

{'AT5G45050.1': 'Q9FL92',
 'AT1G10920.1': 'O04093',
 'AT5G38350.1': 'Q9FF28',
 'AT1G12290.2': 'P60839',
 'AT1G50180.1': 'Q9SX38',
 'AT1G58400.1': 'Q8W3K3',
 'AT5G44510.1': 'Q9FI14',
 'AT1G17600.1': 'F4I901',
 'AT3G07040.1': 'Q39214',
 'AT3G50950.1': 'Q38834',
 'AT5G41740.2': 'F4JYI4',
 'AT5G51630.2': 'Q0WPW2',
 'AT1G61310.1': 'O64789',
 'AT3G44480.1': 'F4J339',
 'AT1G56520.2': 'F4I552',
 'AT1G15890.1': 'Q9LMP6',
 'AT4G16920.1': 'A0A1P8B6H3',
 'AT1G72840.1': 'F4IEZ7',
 'AT4G09430.1': 'F4JK03',
 'AT4G12020.1': 'Q9SZ67',
 'AT3G44670.2': 'F4J361',
 'AT1G10920.2': 'O04093',
 'AT1G12210.1': 'Q8L3R3',
 'AT5G45200.1': 'F4KD45',
 'AT1G12290.1': 'P60839',
 'AT4G16960.1': 'F4JNB9',
 'AT5G43470.2': 'Q8W4J9',
 'AT5G66910.1': 'Q9FKZ0',
 'AT4G19520.1': 'F4JT82',
 'AT1G57650.2': 'F4I847',
 'AT5G48620.1': 'Q9FJK8',
 'AT5G18370.1': 'F4JWM0',
 'AT4G36140.1': 'O65506',
 'AT5G49140.1': 'Q9FH20',
 'AT5G47260.1': 'Q9LVT3',
 'AT3G51570.1': 'Q9SCZ2',
 'AT1G56520.1': 'F4I552',
 'AT5G40060.1': 'F4KFY1',
 'AT1G63

In [102]:
import pickle
pickle.dump(uniprot_d, open("pickles/uniprot_d.pickle", "wb" ))

In [87]:
for df in upIDs:
    entr_l = [entr for entr in df if entr in allupIDs]
    if not len(entr_l):
        print(df)

0    A0A1P8AXZ9
1        F4IFF6
Name: Entry, dtype: object
0    F4HR52
Name: Entry, dtype: object


In [ ]:
for ii in dups:
    print(tairIDs[ii])
    print(upIDs[ii])

In [90]:
for ii in dups:
    print(tairIDs[ii])
    print(upIDs[ii])

AT1G58807.1
0    P0DI16
1    F4IBE4
Name: Entry, dtype: object
AT4G19050.1
0        Q8GYX0
1        P0CB16
2    A0A221J431
Name: Entry, dtype: object
AT1G59124.1
0    P0DI16
1    F4IBE4
Name: Entry, dtype: object
AT1G58807.2
0    P0DI16
1    F4IBE4
Name: Entry, dtype: object
AT1G53350.1
0        P0C8S1
1        Q3ECR3
2    A0A178W974
Name: Entry, dtype: object
AT4G11170.1
0    O82500
1    O82499
Name: Entry, dtype: object
AT1G64070.1
0    F4I594
1    Q6DST1
Name: Entry, dtype: object
AT1G59218.1
0    P0DI18
1    P0DI17
Name: Entry, dtype: object
AT1G58848.1
0    P0DI17
1    P0DI18
2    B5BRD4
Name: Entry, dtype: object
AT1G59218.2
0    P0DI18
1    P0DI17
Name: Entry, dtype: object
AT1G58848.2
0    P0DI18
1    P0DI17
2    B5BRD5
Name: Entry, dtype: object


In [6]:
for ii in range(len(df)):
    print(df.iloc[ii])

annotation    Athaliana_AT1G10920_1
start                           411
score1                       0.7727
score2                       0.6918
score3                       0.9758
score4                       0.5487
score5                       0.9087
score6                       0.4548
score7                       0.9973
score8                       0.5931
score9                       0.7429
p1                                R
p2                                S
p3                                L
p4                                S
p5                                K
b1                              NaN
m1                                S
m2                                R
m3                                R
m4                                L
m5                                S
m6                                V
b2                              NaN
s1                                H
s2                                G
s3                                G
s4                          

In [261]:
#gets pdb structures from pdb database
if False:
    pdbl = PDBList()
    for code in set(df['PDB ID + chain']):
        pdbl.retrieve_pdb_file(code[:4], file_format = 'pdb', pdir='pdb/LRRPredictor')

In [4]:
import warnings
warnings.filterwarnings("ignore")

# edge cases: '2Z66A', 3JB9J, 4CP6A, 2ELLA, '2CA6A'

LRR_d = {}

for ii in range(df.shape[0]):
    if ii%2:
        code = df.iloc[ii]['PDB ID + chain']
        if code != '2CA6A':
            pass
        pdb_code = code[:4]
        chain_id = code[4]
        entry = df.iloc[ii-1]['entry']
        motifs = df.iloc[ii]['entry']        
        print(code)

        prefixSize = 5
        matchingRecords = []
        for record in SeqIO.parse("pdb/LRRPredictor/pdb%s.ent"%pdb_code, "pdb-seqres"):                 
            if record.annotations["chain"].upper()==chain_id:
                delta = 0
                offset = -1
                while delta < 10 and offset < 0:
                    offset = record.seq[delta:].find(entry[delta:prefixSize+delta])
                    delta += 1
                if offset >=0:
                    matchingRecords.append((record, offset))
        if not matchingRecords:
            raise Exception ('did not find chain with matching prefix')
        elif len(matchingRecords)>1:
            raise Exception ('found more than one matching record')
        offset = matchingRecords[0][1]
        chain_id_cased = matchingRecords[0][0].annotations["chain"]       
                
        #get pdb sequence
        parser = PDBParser()
        structure = parser.get_structure(pdb_code, "pdb/LRRPredictor/pdb%s.ent"%pdb_code)        
        chains = []
        for model in structure:
            for chain in model:
                if chain.id == chain_id_cased:
                    chains.append(chain)
                    
        if len(chains) == 0:
            raise Exception ('no chains found')
        elif len(chains) == 1:
            chain = chains[0]
            pdbseq = ''.join([seq1(res.get_resname()) for res in chain.get_residues()])
        else:
            print('found too many chains:', len(chains))
            chain = chains[0]
            pdbseq = ''.join([seq1(res.get_resname()) for res in chain.get_residues()])            
        
        #try to align motif window to pdb sequence

        matrix = matlist.blosum62
        queryRadius = 5
        motifLocs = []
        for jj, char in enumerate(motifs):
            if char != '-':
                if jj-queryRadius<0:
                    lb = 0
                    ub = 2*queryRadius
                elif jj+queryRadius>len(entry):
                    lb = len(entry)-2*queryRadius
                    ub = len(entry)
                else:
                    lb = jj - queryRadius
                    ub = jj + queryRadius
                query = entry[lb:ub]
                
                motifLoc = pdbseq.find(query)+jj-lb
                
                if motifLoc < 0:
                    print('needed to do alignment')
                    print(query)
                    try:
                        a=pairwise2.align.localds(query, pdbseq, matrix, -1, -1, penalize_end_gaps=(False, False), one_alignment_only=True)[0]
                        print(format_alignment(*a))
                        # a = next(pairwise2.align.localdx(query, pdbseq, matrix, one_alignment_only=True))                        
                        # a = next(pairwise2.align.localds(query, pdbseq, matrix, -1, -1, penalize_end_gaps=(False, False), one_alignment_only=True))                        
                    except:
                        print(pdbseq)
                        raise Exception('alignment failed')
                    if a.score <queryRadius*2:
                        raise Exception('no alignments found')
                    
                    queryShift = int(format_alignment(*a).split(' ')[0])-1
                    start = int(format_alignment(*a).split(' ')[3].split('\n')[1])-1
                    motifLoc = start+jj-lb-queryShift
                    print(motifLoc)
                motifLocs.append(motifLoc)

        print(motifLocs)
        LRR_d[code] = motifLocs
    

1A9NA
[19, 42, 64, 88, 113, 140]
1H6UA
[42, 64, 86, 108, 130, 152, 174, 196, 218]
1IO0A
[18, 47, 75, 103, 133]
1JL5A
[39, 59, 79, 99, 121, 141, 163, 183, 205, 225, 245]
1O6VA
[45, 67, 89, 111, 133, 155, 176, 198, 220, 242]
1OGQA
[51, 77, 102, 126, 150, 175, 198, 222, 245]
1OZNA
[12, 33, 57, 81, 106, 130, 154, 178, 202, 226, 254]
1P9AG
[11, 32, 56, 78, 101, 125, 149, 173, 196]
1PGVA
[16, 45, 73, 101, 132]
1W8AA
[9, 30, 55, 79, 103, 127]
1WWLA
[5, 28, 65, 92, 118, 146, 170, 198, 225, 250]
1XEUA
[42, 64, 86, 107, 129, 151, 173]
1XKUA
[11, 32, 56, 80, 101, 125, 151, 172, 196, 220, 243]
1ZIWA
[2, 23, 47, 71, 95, 119, 143, 169, 193, 220, 246]
2ASTB
[42, 65, 88, 113, 137, 163, 190, 217, 242]
2BNHA
[25, 53, 82, 110, 139, 167, 196, 224, 253]
2CA6A
needed to do alignment
MARFSIEGKS
2 ARFSIEGKS
  |||||||||
1 ARFSIEGKS
  Score=43

-1
[-1, 32, 60, 94, 122, 159, 187, 216, 244, 274, 303, 332]
2ELLA
found too many chains: 20
[25, 50, 72, 96, 121, 148]
2FT3A
[10, 31, 55, 79, 100, 124, 149, 170, 194, 21

In [5]:
import pickle

with open('pickles/LRRPred_validation.pickle', 'wb') as handle:
    pickle.dump(LRR_d, handle, protocol = pickle.HIGHEST_PROTOCOL)


In [230]:
del LRR_d